In [ ]:
import numpy as np
import math
import xarray as xr
from scipy.stats import genextreme
import datetime
import matplotlib.pyplot as plt # Assumed installed for example plot
from scipy.spatial.distance import cdist
from GEV import * 
from utils import *

Data Generation functions

In [ ]:
def create_independent_gev_grid(loc, scale, shape,
                                num_x=5, num_y=5, num_t=5):
    """
    Generates an xarray.Dataset with independent GEV samples at each point.

    Each value in the grid (x, y, t) is drawn independently from the same
    GEV(loc, scale, shape) distribution.

    Args:
        loc (float): The location parameter (mu) of the GEV distribution.
        scale (float): The scale parameter (sigma > 0) of the GEV distribution.
        shape (float): The shape parameter (xi) of the GEV distribution.
                       Note: Internally uses c = -shape for scipy.stats.genextreme.
                       shape < 0 (c > 0) => Weibull type (bounded above)
                       shape = 0 (c = 0) => Gumbel type
                       shape > 0 (c < 0) => Fréchet type (heavy tail)
        num_x (int): Number of points along the x dimension. Default is 5.
        num_y (int): Number of points along the y dimension. Default is 5.
        num_t (int): Number of points along the t dimension. Default is 5.

    Returns:
        xarray.Dataset: An xarray Dataset with dimensions ('x', 'y', 't')
                        and a single data variable 'extreme_value' containing
                        independent GEV samples.

    Raises:
        ValueError: If the scale parameter is not positive.
    """
    # --- Input Validation ---
    if scale <= 0:
        raise ValueError("Scale parameter (sigma) must be positive.")

    # --- Define Coordinates ---
    x_coords = np.arange(num_x)
    y_coords = np.arange(num_y)
    t_coords = np.arange(num_t)

    # --- Generate GEV Data ---
    total_values = num_x * num_y * num_t
    # Generate independent samples using the specified parameters
    # Note: Using c = -shape consistent with user input convention
    gev_data_flat = genextreme.rvs(c=-shape, loc=loc, scale=scale, size=total_values)
    gev_data_3d = gev_data_flat.reshape(num_x, num_y, num_t)

    # --- Create xarray DataArray ---
    data_variable = xr.DataArray(
        data=gev_data_3d,
        coords={'lon': x_coords, 'lat': y_coords, 'time': t_coords},
        dims=['lon', 'lat', 'time'],
        name='extreme_value',
        attrs={
            'description': 'Independent synthetic data generated from a GEV distribution.',
            'units': 'unitless',
            'gev_location': loc,
            'gev_scale': scale,
            'gev_shape_xi': shape, # Store the input shape parameter (xi)
            'scipy_shape_c': -shape # Store the c parameter used by scipy
        }
    )
    # --- Create xarray Dataset ---
    dataset = xr.Dataset({'extreme_value': data_variable})
    dataset.attrs['creation_timestamp'] = datetime.datetime.now().isoformat()
    dataset.attrs['source'] = 'Generated by create_independent_gev_grid'
    dataset.attrs['grid_type'] = 'dimensionless integer coordinates'
    dataset.attrs['dependencies'] = 'Identically distributed, independent samples'

    return dataset



def create_gev_random_linear_trend(loc, scale, shape,
                                   num_x=5, num_y=5, num_t=5, seed=0):
    """
    Generates GEV data with a location parameter varying linearly with time.

    The location parameter at time t is loc(t) = b0 + b1 * t.
    The intercept b0 is randomly chosen based on the input 'loc'.
    The slope b1 is randomly chosen based on 'scale' and 'num_t'.
    Both are generated using the specified 'seed'.
    Scale and shape parameters are constant.

    Args:
        loc (float): The base location parameter (mu) around which the trend starts (t=0).
        scale (float): The constant scale parameter (sigma > 0). Used for GEV generation
                       and influences the magnitude of the random trend slope.
        shape (float): The constant shape parameter (xi).
                       Note: Internally uses c = -shape for scipy.stats.genextreme.
        num_x (int): Number of points along the x dimension. Default is 5.
        num_y (int): Number of points along the y dimension. Default is 5.
        num_t (int): Number of points along the t dimension. Default is 5.
        seed (int): Seed for the random number generator to determine b0 and b1. Default is 0.

    Returns:
        xarray.Dataset: An xarray Dataset with dimensions ('x', 'y', 't')
                        containing GEV samples with a time-varying location parameter.
                        The generated b0 and b1 are stored as attributes.

    Raises:
        ValueError: If scale <= 0.
    """
    # --- Input Validation ---
    if scale <= 0:
        raise ValueError("Scale parameter (sigma) must be positive.")

    # --- Generate Trend Parameters Internally ---
    rng = np.random.default_rng(seed) # Use modern generator with seed

    # Generate b0: Centered around 'loc' with slight variation based on 'scale'
    # Ensures loc(t=0) is close to the provided 'loc'
    b0 = rng.normal(loc=loc, scale=abs(scale * 0.1)) # Use abs(scale) just in case

    # Generate b1: Slope determined relative to scale and time duration
    # Aim for total change over the period to be within a few 'scale' units
    # k defines target total change factor (e.g., k=1 means loc changes by ~1*scale over num_t steps)
    k = rng.uniform(0, 10) # Random factor for total change magnitude/direction
    time_horizon = max(1, num_t - 1) # Avoid division by zero if num_t=1
    b1 = (k * scale) / time_horizon # Calculate the resulting slope per time step

    # --- Define Coordinates ---
    x_coords = np.arange(num_x)
    y_coords = np.arange(num_y)
    t_coords = np.arange(num_t)

    # --- Calculate Time-Varying Location Parameter ---
    loc_t = b0 + b1 * t_coords # Shape (num_t,)

    # --- Generate GEV Data (Iterating through time) ---
    gev_data_3d = np.empty((num_x, num_y, num_t))
    num_spatial_points = num_x * num_y

    for i in range(num_t):
        current_loc = loc_t[i]
        # Generate samples for all x,y points at this time step 'i'
        # Note: Using c = -shape consistent with input convention
        time_slice_flat = genextreme.rvs(c=-shape, loc=current_loc, scale=scale,
                                         size=num_spatial_points)
        # Reshape and store in the main array
        gev_data_3d[:, :, i] = time_slice_flat.reshape(num_x, num_y)

    # --- Create xarray DataArray ---
    data_variable = xr.DataArray(
        data=gev_data_3d,
        coords={'lon': x_coords, 'lat': y_coords, 'time': t_coords},
        dims=['lon', 'lat', 'time'],
        name='extreme_value', # New name for variable
        attrs={
            'description': 'Synthetic GEV data with internally generated linear trend in location (loc = b0 + b1*t).',
            'units': 'unitless',
            'trend_intercept_b0': b0, # Store generated b0
            'trend_slope_b1': b1,     # Store generated b1
            'trend_rng_seed': seed,   # Store seed used
            'gev_scale': scale,       # Constant scale
            'gev_shape_xi': shape,    # Constant shape (xi)
        }
    )
    # --- Create xarray Dataset ---
    dataset = xr.Dataset({'extreme_value': data_variable})
    dataset.attrs['creation_timestamp'] = datetime.datetime.now().isoformat()
    dataset.attrs['source'] = 'Generated by create_gev_random_linear_trend'
    dataset.attrs['grid_type'] = 'dimensionless integer coordinates'
    dataset.attrs['dependencies'] = 'Time dependence via loc(t), spatial independence at fixed t.'
    dataset.attrs['trend_slope_b1'] = b1

    return dataset

def create_gev_nonstationary(loc, scale, shape, trend_function,
                           num_x=5, num_y=5, num_t=5, seed=0):
    """
    Generates GEV data with a location parameter varying as a function of x, y, and t.

    The location parameter is defined by a user-provided function string:
    loc(x, y, t) = f(x, y, t), specified by 'trend_function'.
    The function can include polynomials, affine terms, and common mathematical
    functions (sin, cos, exp, log, sqrt) applied to x, y, and t.

    Args:
        loc (float): A base value added to the calculated trend function result.
                     Can be thought of as the overall mean offset.
        scale (float): The constant scale parameter (sigma > 0).
        shape (float): The constant shape parameter (xi).
        trend_function (str): A string representing the function of x, y, and t
            that defines the spatial and temporal trend of the location parameter.
            Examples: 'x', 'y', 't', 'x+y', 'x+y+t', 'x**2 + y*t', '2*x + 3*y - t',
                      'np.sin(x/5) + np.cos(t/10)', 'np.exp(-(x**2 + y**2)/10) * t'.
            Use 'np.' prefix for numpy functions (sin, cos, exp, log, log10, sqrt).
            The string should only contain 'x', 'y', 't', numbers, basic arithmetic
            operators (+, -, *, /), the power operator (**), parentheses, '.',
            and allowed 'np.' functions. No spaces allowed except within function names.
        num_x (int): Number of points along the x dimension. Default is 5.
        num_y (int): Number of points along the y dimension. Default is 5.
        num_t (int): Number of points along the t dimension. Default is 5.
        seed (int): Seed for the random number generator used for GEV sampling. Default is 0.

    Returns:
        xarray.Dataset: An xarray Dataset with dimensions ('lon', 'lat', 'time')
                        containing GEV samples with a spatially and temporally
                        varying location parameter. The input trend function
                        and base loc are stored as attributes.

    Raises:
        ValueError: If scale <= 0 or the trend_function is invalid/unsafe.
        NameError: If the trend_function uses invalid variables or functions.
        SyntaxError: If the trend_function has invalid syntax.
    """
    # --- Input Validation ---
    if scale <= 0:
        raise ValueError("Scale parameter (sigma) must be positive.")

    # --- Validate the trend_function ---
    if not isinstance(trend_function, str):
        raise TypeError("trend_function must be a string.")

    # Enhanced validation: Allow 'x', 'y', 't', digits, operators, parentheses, '.', and 'np.' prefix
    # This is a basic check; eval's safety relies more on the restricted namespace.
    # It aims to prevent obviously malicious code, but isn't foolproof against complex attacks.
    # We explicitly list allowed numpy functions later.
    allowed_pattern = r'^[xy\d\+\-\*\/\(\)\.\*\*npcosinlgqt]*$' # Basic character check
    if not re.match(allowed_pattern, trend_function.replace(' ', '')): # Remove spaces for check
         raise ValueError(
            "Invalid characters in trend_function. Use only 'x', 'y', 't', numbers, "
            "operators (+, -, *, /, **), parentheses, '.', and allowed 'np.' functions "
            "(sin, cos, exp, log, log10, sqrt)."
        )

    # Check for balanced parentheses
    if trend_function.count('(') != trend_function.count(')'):
        raise ValueError("Unbalanced parentheses in trend_function.")

    # --- Define Coordinates ---
    x_coords = np.arange(num_x)
    y_coords = np.arange(num_y)
    t_coords = np.arange(num_t)

    # --- Create meshgrids for x, y, t ---
    x_grid, y_grid, t_grid = np.meshgrid(x_coords, y_coords, t_coords, indexing='ij')

    # --- Calculate Spatially and Temporally Varying Location Parameter ---
    try:
        # Define allowed functions from numpy for the safe namespace
        # Using np directly is safer than importing individual functions into the main scope
        allowed_numpy_funcs = {
            'sin': np.sin,
            'cos': np.cos,
            'exp': np.exp,
            'log': np.log,   # Natural logarithm
            'log10': np.log10, # Base-10 logarithm
            'sqrt': np.sqrt,
            # Add other safe numpy functions here if needed
        }

        # Create local dictionary for safe evaluation
        local_namespace = {
            'x': x_grid,
            'y': y_grid,
            't': t_grid,
            'np': np, # Provide access to the numpy module itself
             # Add safe math constants if desired
            'pi': math.pi
        }
        
        # Check if the trend function tries to use disallowed attributes/functions of np
        # This is an additional safety layer against accessing things like np.load, np.eval, etc.
        potential_np_calls = re.findall(r'np\.(\w+)', trend_function)
        for func_name in potential_np_calls:
            if func_name not in allowed_numpy_funcs:
                 # Check if it's a known numpy constant/attribute we might allow (like pi)
                 # For simplicity, we are currently strict and only allow functions defined above.
                 # If you need constants like np.pi, add 'pi': np.pi to allowed_numpy_funcs
                 # or handle them separately. Here we raise an error.
                 raise NameError(f"Disallowed numpy function/attribute used: np.{func_name}. Allowed functions: {list(allowed_numpy_funcs.keys())}")


        # Evaluate the trend function using the safe namespace
        # The base 'loc' is added *after* evaluating the trend function.
        trend_values = eval(trend_function, {"__builtins__": None}, local_namespace)

        # Add the base location value
        loc_xyt = loc + trend_values

        # Validate the shape of the result
        expected_shape = (num_x, num_y, num_t)
        if not isinstance(loc_xyt, np.ndarray) or loc_xyt.shape != expected_shape:
            # Handle scalar results (e.g., if trend_function is just a number)
            if np.isscalar(loc_xyt):
                 loc_xyt = np.full(expected_shape, float(loc_xyt))
            else:
                raise ValueError(
                    f"The trend_function '{trend_function}' did not produce the correct shape "
                    f"{expected_shape}. Resulting shape was {getattr(loc_xyt, 'shape', type(loc_xyt))}."
                 )

    except NameError as e:
        # Catch undefined variables or functions used within the eval string
        raise NameError(f"Invalid variable or function used in trend_function: {e}. Ensure 'np.' prefix for numpy funcs.") from e
    except SyntaxError as e:
        raise SyntaxError(f"Syntax error in trend_function: {e}") from e
    except TypeError as e:
        # Catches errors like applying math functions to incompatible types if eval somehow allows it
        raise TypeError(f"Type error during trend_function evaluation: {e}") from e
    except Exception as e:
        # Catch-all for other potential errors during eval
        raise ValueError(f"Error evaluating trend_function '{trend_function}': {e}") from e

    # --- Generate GEV Data ---
    # Use the pre-calculated loc_xyt which has the correct shape
    # Generate random numbers once using the broadcasted loc_xyt
    rng = np.random.default_rng(seed)
    gev_data_3d = genextreme.rvs(c=-shape, loc=loc_xyt, scale=scale, size=loc_xyt.shape, random_state=rng)

    # --- Create xarray DataArray ---
    data_variable = xr.DataArray(
        data=gev_data_3d,
        coords={'lon': x_coords, 'lat': y_coords, 'time': t_coords},
        dims=['lon', 'lat', 'time'],
        name='extreme_value',
        attrs={
            'description': 'Synthetic GEV data with non-stationary location parameter varying in x, y, and t.',
            'units': 'unitless',
            'trend_function': trend_function,
            'base_location_offset': loc, # Store the base loc value
            'gev_scale': scale,
            'gev_shape_xi': shape,
            'gev_rng_seed': seed, # Renamed seed attribute for clarity
        }
    )

    # --- Create xarray Dataset ---
    dataset = xr.Dataset({'extreme_value': data_variable})
    dataset.attrs['creation_timestamp'] = datetime.datetime.now().isoformat()
    dataset.attrs['source'] = 'Generated by create_gev_nonstationary'
    dataset.attrs['grid_type'] = 'dimensionless integer coordinates'
    dataset.attrs['dependencies'] = 'Spatio-temporal dependence via user-defined loc(x,y,t) = base_loc + trend_function(x,y,t).'

    return dataset

def create_max_stable_process_grid(num_x=3, num_y=3, num_t=3, n_spectra=10, range_param=1, loc=68, scale=11.8, shape=0.1):
    """
    Simulate a max-stable process (Brown-Resnick type) over a spatial-temporal grid
    and normalize its marginals to a specified GEV distribution.
    
    Args:
        num_x (int): Number of grid points along x (longitude).
        num_y (int): Number of grid points along y (latitude).
        num_t (int): Number of time steps.
        n_spectra (int): Number of spectral functions used in approximation (more = more accurate).
        range_param (float): Range/scale parameter controlling spatial dependence.
        loc (float): The location parameter for the GEV distribution.
        scale (float): The scale parameter for the GEV distribution.
        shape (float): The shape parameter for the GEV distribution.
        
    Returns:
        xarray.Dataset: Max-stable process on a 3D grid (lon, lat, time) with normalized marginals.
    """
    
    # --- Grid Setup ---
    x_coords = np.arange(num_x)
    y_coords = np.arange(num_y)
    t_coords = np.arange(num_t)
    lon, lat = np.meshgrid(x_coords, y_coords, indexing='ij')
    spatial_locs = np.column_stack((lon.ravel(), lat.ravel()))  # Flattened (num_points, 2)

    # --- Gaussian Process Covariance ---
    def brown_resnick_covariance(h, range_param=1.0):
        return np.exp(-h / range_param)

    num_points = spatial_locs.shape[0]
    distances = cdist(spatial_locs, spatial_locs)
    cov_matrix = brown_resnick_covariance(distances, range_param)

    # --- Generate Spectral Functions ---
    max_field = np.full((num_x * num_y, num_t), -np.inf)
    for i in range(n_spectra):
        # Sample from a multivariate Gaussian process
        gp_sample = np.random.multivariate_normal(mean=np.zeros(num_points), cov=cov_matrix)
        gp_sample = gp_sample.reshape(num_x, num_y)

        for t in range(num_t):
            w_t = np.random.normal(0, 1)  # Simple white noise in time
            z = np.exp(gp_sample + w_t)  # Log-Gaussian
            max_field[:, t] = np.maximum(max_field[:, t], np.random.exponential() * z.ravel())

    # --- Reshape max_field into a 3D grid ---
    max_field = max_field.reshape(num_x, num_y, num_t)

    # --- Normalize to the desired GEV distribution ---
    normalized_field = np.zeros_like(max_field)
    
    for i in range(num_x):
        for j in range(num_y):
            # Convert max-stable values to uniform (Unit Fréchet)
            u = np.exp(-1 / max_field[i, j, :])  # Unit Fréchet transformation (i.e., converting max-stable to uniform)
            
            # Now transform uniform values to Standard Fréchet (location=0, scale=1, shape=1)
            normalized_field[i, j, :] = genextreme.ppf(u, c=-0.1, loc=68, scale=10)
    
    # --- Package into xarray ---
    data_variable = xr.DataArray(
        data=normalized_field,
        coords={'lon': x_coords, 'lat': y_coords, 'time': t_coords},
        dims=['lon', 'lat', 'time'],
        name='extreme_value',
        attrs={
            'description': 'Synthetic max-stable process (Brown–Resnick approximation) with normalized marginals',
            'units': 'unitless',
            'type': 'max-stable',
            'n_spectra': n_spectra,
            'range_param': range_param,
            'loc': loc,
            'scale': scale,
            'shape': shape
        }
    )

    dataset = xr.Dataset({'extreme_value': data_variable})
    dataset.attrs['creation_timestamp'] = datetime.datetime.now().isoformat()
    dataset.attrs['source'] = 'Generated by create_max_stable_process_grid'
    dataset.attrs['grid_type'] = 'dimensionless integer coordinates'
    dataset.attrs['process_type'] = 'Brown–Resnick max-stable (spectral representation)'

    return dataset

test = create_max_stable_process_grid(num_x=3,num_y=3,num_t=3)
test


<xarray.Dataset> Size: 252B
Dimensions:        (lon: 3, lat: 3, time: 3)
Coordinates:
  * lon            (lon) int32 12B 0 1 2
  * lat            (lat) int32 12B 0 1 2
  * time           (time) int32 12B 0 1 2
Data variables:
    extreme_value  (lon, lat, time) float64 216B 93.99 95.44 ... 109.0 119.2
Attributes:
    creation_timestamp:  2025-04-14T11:30:28.220726
    source:              Generated by create_max_stable_process_grid
    grid_type:           dimensionless integer coordinates
    process_type:        Brown–Resnick max-stable (spectral representation)

Data visualization functions

In [107]:
def plot_time(data, x_index, y_index):
    """
    Plots the time series for a specific (x, y) point in the xarray data.

    Args:
        data (xarray.Dataset or xarray.DataArray): The dataset or data array
            containing the 'extreme_value' variable with dims ('x', 'y', 't').
        x_index (int): The integer index for the x dimension.
        y_index (int): The integer index for the y dimension.
    """
    # If input is a Dataset, extract the DataArray
    if isinstance(data, xr.Dataset):
        if 'extreme_value' not in data:
            print("Error: Dataset does not contain 'extreme_value' variable.")
            return
        da = data['extreme_value']
    elif isinstance(data, xr.DataArray):
        da = data
    else:
        print("Error: Input must be an xarray Dataset or DataArray.")
        return

    try:
        # Select the data point using integer indices
        selected_point_data = da.isel(x=x_index, y=y_index)

        # Create the plot
        plt.figure(figsize=(10, 5)) # Adjust figure size if needed
        selected_point_data.plot.line(marker='.', linestyle='-') # Use xarray's plotting

        # Customize
        plt.title(f"Time Series at x={selected_point_data['x'].item()}, y={selected_point_data['y'].item()}")
        plt.xlabel("Dimensionless Time (t)")
        plt.ylabel(da.name or "Value") # Use variable name if available
        plt.grid(True, alpha=0.5)
        plt.tight_layout() # Adjust layout
        plt.show()

    except IndexError:
        print(f"Error: Indices x={x_index}, y={y_index} are out of bounds "
              f"for data with shape {da.shape[:2]}.")
    except Exception as e:
        print(f"An error occurred during time series plotting: {e}")


def map(data, t_index):
    """
    Plots a 2D spatial map (image) of the data at a specific time index 't'.

    Args:
        data (xarray.Dataset or xarray.DataArray): The dataset or data array
            containing the 'extreme_value' variable with dims ('x', 'y', 't').
        t_index (int): The integer index for the t dimension.
    """
    # If input is a Dataset, extract the DataArray
    if isinstance(data, xr.Dataset):
        if 'extreme_value' not in data:
            print("Error: Dataset does not contain 'extreme_value' variable.")
            return
        da = data['extreme_value']
    elif isinstance(data, xr.DataArray):
        da = data
    else:
        print("Error: Input must be an xarray Dataset or DataArray.")
        return

    try:
        # Select the time slice using integer index
        selected_time_slice = da.isel(t=t_index)

        # Create the plot using imshow
        plt.figure(figsize=(8, 6)) # Adjust figure size if needed
        selected_time_slice.plot.imshow(cmap='viridis') # Use xarray's imshow plotting

        # Customize
        # Xarray's plot.imshow usually sets title and labels reasonably
        plt.title(f"Spatial Map at t = {selected_time_slice['t'].item()}")
        plt.xlabel("Dimensionless Coordinate (x)")
        plt.ylabel("Dimensionless Coordinate (y)")
        # Colorbar is added automatically by default with plot.imshow
        plt.tight_layout()
        plt.show()

    except IndexError:
        print(f"Error: Time index t={t_index} is out of bounds "
              f"for data with time dimension size {da.shape[2]}.")
    except Exception as e:
        print(f"An error occurred during spatial map plotting: {e}")
        

In [138]:
location_param = 68
scale_param = 11.8
shape_param = 0.1

# 2. Specify grid dimensions
x_dim_size = 10
y_dim_size = 10
t_dim_size = 10

# 3. Generate the xarray dataset
try:
    gev_dataset_dimless = create_gev_random_linear_trend(
        loc=location_param,
        scale=scale_param,
        shape=shape_param,
        num_x=x_dim_size,
        num_y=y_dim_size,
        num_t=t_dim_size
    )

    # 4. Inspect the dataset (optional)
    print("Generated xarray Dataset (Dimensionless Coordinates):")
    print(gev_dataset_dimless)

    # 5. Use the new plotting functions
    # --- Plot time series at a specific point (e.g., x=10, y=15) ---
    plot_x_index = 5
    plot_y_index = 5
    print(f"\nPlotting time series for point (lon={plot_x_index}, lat={plot_y_index})...")
    plot_time(gev_dataset_dimless, x_index=plot_x_index, y_index=plot_y_index)

    # --- Plot spatial map at a specific time (e.g., t=0) ---
    plot_t_index = 0
    print(f"\nPlotting spatial map for time index (time={plot_t_index})...")
    map(gev_dataset_dimless, t_index=plot_t_index)

    # --- Example: Plot spatial map at a later time (e.g., t=50) ---
    plot_t_index_later = 50
    print(f"\nPlotting spatial map for time index (time={plot_t_index_later})...")
    map(gev_dataset_dimless, t_index=plot_t_index_later)


except ValueError as e:
    # Handles the ValueError check for scale <= 0 inside the function
    print(f"Error generating dataset: {e}")

gev_dataset_dimless

Generated xarray Dataset (Dimensionless Coordinates):
<xarray.Dataset> Size: 8kB
Dimensions:        (lon: 10, lat: 10, time: 10)
Coordinates:
  * lon            (lon) int32 40B 0 1 2 3 4 5 6 7 8 9
  * lat            (lat) int32 40B 0 1 2 3 4 5 6 7 8 9
  * time           (time) int32 40B 0 1 2 3 4 5 6 7 8 9
Data variables:
    extreme_value  (lon, lat, time) float64 8kB 68.67 73.61 ... 114.0 92.92
Attributes:
    creation_timestamp:  2025-04-14T11:38:44.190699
    source:              Generated by create_gev_random_linear_trend
    grid_type:           dimensionless integer coordinates
    dependencies:        Time dependence via loc(t), spatial independence at ...
    trend_slope_b1:      3.537203580459633

Plotting time series for point (lon=5, lat=5)...
An error occurred during time series plotting: Dimensions {'y', 'x'} do not exist. Expected one or more of ('lon', 'lat', 'time')

Plotting spatial map for time index (time=0)...
An error occurred during spatial map plotting: Dimensio

<xarray.Dataset> Size: 8kB
Dimensions:        (lon: 10, lat: 10, time: 10)
Coordinates:
  * lon            (lon) int32 40B 0 1 2 3 4 5 6 7 8 9
  * lat            (lat) int32 40B 0 1 2 3 4 5 6 7 8 9
  * time           (time) int32 40B 0 1 2 3 4 5 6 7 8 9
Data variables:
    extreme_value  (lon, lat, time) float64 8kB 68.67 73.61 ... 114.0 92.92
Attributes:
    creation_timestamp:  2025-04-14T11:38:44.190699
    source:              Generated by create_gev_random_linear_trend
    grid_type:           dimensionless integer coordinates
    dependencies:        Time dependence via loc(t), spatial independence at ...
    trend_slope_b1:      3.537203580459633

Fitting

In [139]:
#print(gev_dataset_dimless.attrs["trend_slope_b1"])
endog, exog, mdt = xarray_to_endog_exog(ds = gev_dataset_dimless,endog_var="extreme_value",include_time_coords=True, include_space_coords=False)

In [140]:
print(exog.shape)
afit = GEVSample(endog=endog,exog=exog).fit(fit_method='MLE')
print(afit)

(10, 1, 100)

          EVT Results Summary       
Data dimensions : (10, 100)
AIC: 8217.73  
BIC: 8219.54  

+------------+--------------+--------------+--------------+---------------------------+----------+
| Parameter  | Estimate     | Z-score      | P>|Z-score|  | 95% CI                    | Signif   |
+------------+--------------+--------------+--------------+---------------------------+----------+
| μ_0        | 68.6873      | 94.10        | 0.0000       | (67.2567, 70.1179)        | ***      |
| μ_1        | 3.4787       | 24.95        | 0.0000       | (3.2054, 3.7520)          | ***      |
| σ_0        | 11.4120      | 20.69        | 0.0000       | (10.3308, 12.4932)        | ***      |
| σ_1        | 0.1761       | 1.64         | 0.1003       | (-0.0339, 0.3862)         |          |
| ξ_0        | 0.0404       | 4.84         | 0.0000       | (0.0240, 0.0567)          | ***      |
| ξ_1        | 0.0003       | nan          | nan          | (nan, nan)                |          |

c:\github\EVT_Univariate\EVT_Classes\GEV.py:591: RuntimeWarning: invalid value encountered in sqrt
  se = np.sqrt(np.diag(cov_matrix))  # Standard errors
